In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [5]:
import os
import urllib.request as request
import zipfile
from CNN_Classifier import logger
from CNN_Classifier.utils.common import get_size

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-11 16:12:23,586: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-11 16:12:23,594: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-11 16:12:23,596: INFO: common: created directory at: artifacts]
[2024-10-11 16:12:23,596: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-11 16:12:28,785: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 26064254
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2f1d589cb025f2dc67cdc58a4f61f33be6927a40e286469a56aa9eebde9ecddf"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F7F4:2C2A46:19BF3B:20BE82:6709010E
Accept-Ranges: bytes
Date: Fri, 11 Oct 2024 10:42:26 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10236-MAA
X-Cache: 